In [8]:
import ray

# Start Ray. If you're connecting to an existing cluster, you would use
# ray.init(address=<cluster-address>) instead.
ray.shutdown()
ray.init()

2021-12-05 13:59:32,015	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-05 13:59:35,615	ERROR import_thread.py:89 -- ImportThread: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 870, in run


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:59325',
 'raylet_socket_name': 'tcp://127.0.0.1:56567',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\James\\AppData\\Local\\Temp\\ray\\session_2021-12-05_13-59-25_570195_2912',
 'metrics_export_port': 62544,
 'node_id': 'bdeb0a06ccc564d67016ca7f211418a46ebc33ebf51e8a1d5f794ab2'}

    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\worker.py", line 439, in print_logs
    pubsub_client.subscribe(gcs_utils.LOG_FILE_CHANNEL)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1377, in subscribe
    ret_val = self.execute_command('SUBSCRIBE', *new_channels.keys())
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1253, in execute_command
    self.connection = self.connection_pool.get_connection(
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\connection.py", line 1179, in get_connection
    if connection.can_read():
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\connection.py", line 743, in can_read
    return self._parser.can_read(timeout)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\connection.py", line 410, in can_read
    return self.read_from_socket(timeout=timeout,
  File "D:\anaconda3\envs\yourenvname\lib\

In [9]:
ray.is_initialized() 

True

Actors

In [14]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value

counter_actor = Counter.remote()
ray.get(counter_actor.increment.remote())

1

In [15]:
@ray.remote
class Foo(object):

    # Any method of the actor can return multiple object refs.
    @ray.method(num_returns=2)
    def bar(self):
        return 1, 2

f = Foo.remote()
obj_ref1, obj_ref2 = f.bar.remote()
ray.get(obj_ref1)

1

In [16]:
@ray.remote(num_cpus=2, num_gpus=1)
class GPUActor(object):
    pass

@ray.remote(num_cpus=4)
class Counter(object):
    pass

a1 = Counter.options(num_cpus=1, resources={"Custom1": 1}).remote()
a2 = Counter.options(num_cpus=2, resources={"Custom2": 1}).remote()
a3 = Counter.options(num_cpus=3, resources={"Custom3": 1}).remote()

In [ ]:
ray.actor.exit_actor()

In [18]:
import time

@ray.remote
def f(counter):
    for _ in range(1000):
        time.sleep(0.1)
        counter.increment.remote()
        
counter = Counter.remote()
# Start some tasks that use the actor.
[f.remote(counter) for _ in range(3)]
# Print the counter value.
for _ in range(10):
    time.sleep(1)
    print(ray.get(counter.get_counter.remote()))

0
10
19
39
59
89


KeyboardInterrupt: 

In [21]:
import ray

@ray.remote
class Actor:
    pass

# driver_1.py
# Job 1 creates an actor, "orange" in the "colors" namespace.
ray.init(address="auto", namespace="colors",ignore_reinit_error=True)
Actor.options(name="orange", lifetime="detached")

# driver_2.py
# Job 2 is now connecting to a different namespace.
ray.init(address="auto", namespace="fruit",ignore_reinit_error=True)
# This fails because "orange" was defined in the "colors" namespace.
ray.get_actor("orange")

# driver_3.py
# Job 3 connects to the original "colors" namespace
ray.init(address="auto", namespace="colors", ignore_reinit_error=True)
# This returns the "orange" actor we created in the first job.
ray.get_actor("orange")

2021-12-05 14:09:00,084	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379
2021-12-05 14:09:00,087	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
2021-12-05 14:09:00,824	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379
2021-12-05 14:09:00,825	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


ValueError: Failed to look up actor with name 'orange'. This could because 1. You are trying to look up a named actor you didn't create. 2. The named actor died. 3. You did not use a namespace matching the namespace of the actor.

In [22]:
counter = Counter.options(name="CounterActor", lifetime="detached").remote()
counter = ray.get_actor("CounterActor")
print(ray.get(counter.get_counter.remote()))

2021-12-05 14:09:11,697	WARNING worker.py:1245 -- It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="023ddc17-a32a-4ceb-b0f9-c1d1ae42ef3c", ...)


KeyboardInterrupt: 

In [26]:
from ray.util import ActorPool

@ray.remote
class Actor:
    def double(self, n):
        return n * 2

a1, a2 = Actor.remote(), Actor.remote()
pool = ActorPool([a1, a2])

# pool.map(..) returns a Python generator object ActorPool.map
gen = pool.map(lambda a, v: a.double.remote(v), [1, 2, 3, 4])
print([v for v in gen])

[2, 4, 6, 8]


Namespaces

In [27]:
ray.shutdown()
ray.init(namespace="hello")
# or using ray client
#ray.init("ray://<head_node_host>:10001", namespace="world")

2021-12-05 14:11:38,124	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-05 14:11:41,775	ERROR import_thread.py:89 -- ImportThread: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread ray_listen_error_messages:
Traceback (most recent call last):


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:61777',
 'raylet_socket_name': 'tcp://127.0.0.1:61624',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\James\\AppData\\Local\\Temp\\ray\\session_2021-12-05_14-11-31_497014_2912',
 'metrics_export_port': 56978,
 'node_id': 'ee8dfcc48a85aeeb276268f4761f5ad84800a82706255b5c102c2544'}

  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
    self.run()
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 870, in run
  File "D:\anaconda3\envs\yourenvname\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\worker.py", line 439, in print_logs
    pubsub_client.subscribe(gcs_utils.LOG_FILE_CHANNEL)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1377, in subscribe
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\ray\worker.py", line 1211, in listen_error_messages_raylet
    ret_val = self.execute_command('SUBSCRIBE', *new_channels.keys())
  File "D:\anaconda3\envs\yourenvname\lib\site-packages\redis\client.py", line 1253, in execute_command
        worker.error_message_pubsub_client.psubscribe(error_pubsub_channel)
  File "D:\anaconda3\envs\yo

In [30]:
# `ray start --head` has been run to launch a local cluster.
import ray

@ray.remote
class Actor:
    pass

# Job 1 creates two actors, "orange" and "purple" in the "colors" namespace.
with ray.init(namespace="colors", ignore_reinit_error=True):
    Actor.options(name="orange", lifetime="detached")
    Actor.options(name="purple", lifetime="detached")

# Job 2 is now connecting to a different namespace.
with ray.init(namespace="fruits", ignore_reinit_error=True):
  # This fails because "orange" was defined in the "colors" namespace.
    ray.get_actor("orange")
  # This succceeds because the name "orange" is unused in this namespace.
    Actor.options(name="orange", lifetime="detached")
    Actor.options(name="watermelon", lifetime="detached")

# Job 3 connects to the original "colors" namespace
context = ray.init("ray://localhost:10001", namespace="colors", ignore_reinit_error=True)
# This fails because "watermelon" was in the fruits namespace.
ray.get_actor("watermelon")
# This returns the "orange" actor we created in the first job, not the second.
ray.get_actor("orange")
context.disconnect()
# We are manually managing the scope of the connection in this example.

2021-12-05 17:42:53,732	INFO worker.py:852 -- Calling ray.init() again after it has already been called.


AttributeError: __enter__

In [34]:
import ray

@ray.remote
class Actor:
    pass

# Job 1 connects to an anonymous namespace by default
ray.init(ignore_reinit_error=True)
Actor.options(name="my_actor", lifetime="detached")
print(ray.get_runtime_context().namespace)
ray.shutdown()

# Job 2 connects to a _different_ anonymous namespace by default
ray.init(ignore_reinit_error=True)
# This succeeds because the second job is in its own namespace.
Actor.options(name="my_actor", lifetime="detached")
ray.shutdown()

2021-12-05 17:45:05,863	INFO worker.py:852 -- Calling ray.init() again after it has already been called.
2021-12-05 17:45:26,124	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Dependencies

In [40]:
import ray
import requests

# This example runs on a local machine, but you can also do
# ray.init(address=..., runtime_env=...) to connect to a cluster.
ray.shutdown()
ray.init(runtime_env={"pip": ["requests"]})

@ray.remote
def reqs():
    return requests.get("https://www.ray.io/")

print(ray.get(reqs.remote())) # <Response [200]>

2021-12-05 17:49:00,073	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


NotImplementedError: The 'pip' field in runtime_env is not currently supported on Windows.

AsyncIO 

In [41]:
import ray
import asyncio

ray.shutdown()
ray.init()

@ray.remote
class AsyncActor:
    # multiple invocation of this method can be running in
    # the event loop at the same time
    async def run_concurrent(self):
        print("started")
        await asyncio.sleep(2) # concurrent workload here
        print("finished")

actor = AsyncActor.remote()

# regular ray.get
ray.get([actor.run_concurrent.remote() for _ in range(4)])

# async ray.get
await actor.run_concurrent.remote()

2021-12-05 17:49:50,196	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


 pid=14712) started
 pid=14712) started
 pid=14712) started
 pid=14712) started
 pid=14712) finished
 pid=14712) finished
 pid=14712) finished
 pid=14712) finished
 pid=14712) started
 pid=14712) finished


In [42]:
@ray.remote
def some_task():
    return 1

await some_task.remote()
await asyncio.wait([some_task.remote()])

({<Task finished name='Task-5' coro=<_wrap_awaitable() done, defined at D:\anaconda3\envs\yourenvname\lib\asyncio\tasks.py:688> result=1>},
 set())

In [45]:
refs = [some_task.remote() for _ in range(4)]
futs = [ref.future() for ref in refs]
for fut in concurrent.futures.as_completed(futs):
    assert fut.done()
    print(fut.result())

NameError: name 'concurrent' is not defined

In [46]:
import asyncio

@ray.remote
class AsyncActor:
    async def run_task(self):
        print("started")
        await asyncio.sleep(1) # Network, I/O task here
        print("ended")

actor = AsyncActor.remote()
# All 50 tasks should start at once. After 1 second they should all finish.
# they should finish at the same time
ray.get([actor.run_task.remote() for _ in range(50)])

 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) started
 pid=5260) ended
 pid=5260) ended
 pid=5260) ended

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [47]:
import asyncio

@ray.remote
class AsyncActor:
    async def run_task(self):
        print("started")
        await asyncio.sleep(1) # Network, I/O task here
        print("ended")

actor = AsyncActor.options(max_concurrency=10).remote()

# Only 10 tasks will be running concurrently. Once 10 finish, the next 10 should run.
ray.get([actor.run_task.remote() for _ in range(50)])

 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) started
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended
 pid=7500) ended


In [48]:
@ray.remote
class ThreadedActor:
    def task_1(self): print("I'm running in a thread!")
    def task_2(self): print("I'm running in another thread!")

a = ThreadedActor.options(max_concurrency=2).remote()
ray.get([a.task_1.remote(), a.task_2.remote()])

[None, None]

 pid=14660) I'm running in a thread!
 pid=14660) I'm running in another thread!
